In [2]:
import re
import string

import markdown
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame

# merge the train and test csvs

train_csv = pd.concat([pd.read_csv('train/good.csv'), pd.read_csv('train/bad.csv')], axis=0).reset_index()
test_csv = pd.concat([pd.read_csv('test/good.csv'), pd.read_csv('test/bad.csv')], axis=0).reset_index()
# train_csv
# pd.read_csv('test/bad.csv').shape[0]
# pd.read_csv('test/bad.csv').columns

In [3]:



def add_size_columns(csv):
    lengths = []
    for i, row in csv.iterrows():
        lengths.append(len(str(row.readme)))
    csv.insert(2, 'length', lengths)
    return csv


add_size_columns(train_csv)
add_size_columns(test_csv)


# contains 'id', 'readme', 'is_good'

,index,id,length,readme,is_good
0,0,1,4024,# Beego [![Test](https://github.com/beego/beeg...,True
1,1,2,3545,"<p align=""center""><a href=""https://laravel.com...",True
2,2,3,6061,# ![TodoMVC](media/logo.png)\n\n> Helping you ...,True
3,3,4,28301,# Sortable &nbsp; [![Financial Contributors on...,True
4,4,5,13584,"<p align=""center"">\n <a href=""#""><img src=""./...",True
...,...,...,...,...,...
2591,1139,1140,93,# 50projects_50days\n 50 unique mini-projects ...,False
2592,1140,1141,405,# KMG-CAFE\nFIRST PROJECTS\nThis project based...,False
2593,1141,1142,37,# srs\nAutomating services to users\n,False
2594,1142,1143,16,# htmlsitetest\n,False


In [4]:
def create_vocabulary(preprocessed_dataframe: DataFrame) -> set:
    vocabulary = set()
    for i, row in preprocessed_dataframe.iterrows():
        for j in row.readme:
            vocabulary.add(j)
    return vocabulary


In [5]:
def preprocess_html(html):
    # Use BeautifulSoup to parse the HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Remove any script or style tags
    for tag in soup(['script', 'style']):
        tag.decompose()

    # Remove HTML tags and attributes
    text = soup.get_text()
    text = re.sub(r'<[^>]*>', '', text)
    text = text.replace('/', ' ').replace('\\n', ' ').replace('\\r', ' ')
    text = re.sub(r'\\x..', '', text)
    text = text.translate(text.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    text = text.lower()

    # Split the text into words
    words = text.split()
    if len(words) >= 1 and words[0] == 'b':
        del words[0]

    # Return the preprocessed text
    return words


count = 0


def preprocess_markdown(markdown_text):
    markdown_text = str(markdown_text)
    html = markdown.markdown(markdown_text)
    # Preprocess the HTML
    words = preprocess_html(html)
    return words


def preprocess(csv):
    a = csv['readme'].apply(preprocess_markdown)
    return a


train_csv['readme'] = preprocess(train_csv)

train_csv
# test_csv = preprocess(test_csv)
# for i, row in train_csv.iterrows():
#     print(len(row.readme))
#     break


,index,id,length,readme,is_good
0,0,1,6051,"[babylon, js, getting, started, play, directly...",True
1,1,2,13906,"[java, pencil2, computer, cloud, art, floppy, ...",True
2,2,3,35891,"[banner, 2, 0, ui, banner, 2, 0, banner, 1, 4,...",True
3,3,4,4654,"[install, documentation, releases, contributin...",True
4,4,5,3121,"[metasploit, the, metasploit, framework, is, r...",True
...,...,...,...,...,...
2560,1070,1071,301,[],False
2561,1071,1072,1502,"[hi, i, m, kyle, a, full, stack, javascript, d...",False
2562,1072,1073,71,"[hello, world, hello, world, stuff, hello, thi...",False
2563,1073,1074,1018,[],False


In [6]:
test_csv['readme'] = preprocess(test_csv)

test_csv


,index,id,length,readme,is_good
0,0,1,4024,"[beego, beego, is, used, for, rapid, developme...",True
1,1,2,3545,"[about, laravel, note, this, repository, conta...",True
2,2,3,6061,"[helping, you, select, an, mv, framework, webs...",True
3,3,4,28301,"[sortable, sortable, is, a, javascript, librar...",True
4,4,5,13584,"[a, completely, customizable, framework, for, ...",True
...,...,...,...,...,...
2591,1139,1140,93,"[50projects, 50days, 50, unique, mini, project...",False
2592,1140,1141,405,"[kmg, cafe, first, projects, this, project, ba...",False
2593,1141,1142,37,"[srs, automating, services, to, users]",False
2594,1142,1143,16,[htmlsitetest],False


In [7]:
vocabulary = create_vocabulary(train_csv)
# vocabulary
# train_readmes = train_csv
# test_readmes = test_csv

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer


def input(csv):
    res = list(csv['readme'])
    for i in range(len(res)):
        res[i] = ' '.join(res[i])
    return res


train_vectors = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform(input(train_csv))
test_vectors = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform(input(test_csv))
# print(vectors.toarray())
# vectors = vectorize(good_train_csv['readme'])
# for i in vectors.toarray()[0]:
#     print(i)

In [56]:

from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from scipy.sparse import csr_matrix

train_length = csr_matrix(train_csv.length).transpose()
test_length = csr_matrix(test_csv.length).transpose()
train_features = hstack((train_vectors, train_length))
test_features = hstack((test_vectors, test_length))
model = RandomForestClassifier()
model.fit(train_features, train_csv.is_good)
model.score(test_features, test_csv.is_good)


0.9426040061633282

In [60]:
test_readme = """
# readme-classifier
this is a classifier that is used to determine whether a github readme file is "good" or "bad"

this idea behind this project was to change the static classifier found in [DevOpsify](https://github.com/oubaydos/DevOpsify)

### score
for now the score for this model is *93%*, but i think there could be a problem of overfitting, i will do more tests

### example
when given this Readme
```python
test_readme = """
#this readme
"""
```
it gives
```python
print(predict(test_readme))
# False
```
so apperently this is a bad readme 😂
"""


def predict(readme: str) -> bool:
    vector_html = markdown.markdown(readme)
    preprocessed_test = preprocess_html(vector_html)
    vector_test = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform([' '.join(preprocessed_test)])
    return bool(model.predict(hstack((vector_test, len(readme))))[0])


print(predict(test_readme))

False
